#  Project Description

##  Background

Eye tracking is a technology that is used to measure the movement and position of the eye. Eye tracking can be used to obtain a variety of information, such as where someone is looking (also known as the gaze point). The raw eye tracking data cann also be used to engineer new features - eye tracking events - which can further be used to obtain more information. 

The types of eye tracking events that we can measure for include fixations, which are periods of time where the eye fixates on a target. There are saccades where the eyes move between points of fixations. There are also post-saccidic oscillations and glissades where the eye will oscillate after a saccade before settling to a fixation point. Post-saccadic oscillations overshoot the target, while glissades undershoot.

These types of events can be measured by applying different threshold techniques. I-VT applies a velocity threshold; If the speed between two gaze points is below a certain threshold, it is identified as a fixation. If the speed is above the threshold, it is a  saccade. There is also a dispersion/distance based method as well known as I-DT, that uses the distance between the gaze points instead to classify either fixations and saccades. These threshold algorithms are common in practice, but do not have the ability to classify more complex events. 

For the purpose of performing the I-VT  algorithm, a speed of 0.5px/ms was selected, and a dispersion of 1º was selected for I-DT.

## Dataset

For the following notebook, the dataset used is from a study performed in the University of Guelph DRiVE lab. Particpants wore eye-tracking glasses (Tobii Pro 3 glasses) and drove an OKTAL driving simulator. The dataset contains 74 participants that are randomly separated into train, test and validation sets. This will prevent leakage amonst the different particpant data.  Each of the files contains 3 different sets of data. There is some device information that is read in and in the sheet titled 'Event Data'. There is IMU sensor data in the sheet titled 'IMU Data'. The eye tracking data is in the sheet titled 'Gaze Data'. The sheets have 4, 22 and 11 columns respectively. The data  from the eye tracker is collected at 60Hz, and each participant file has roughly 20000 records in each file. The data is pre-split to ensure that there is no leakage between participant data, which could affect the training of the models, and to ensure a more consistent evaluation of the performance of the models.

## Procedure

1. The Gaze Data is read into the notebook using an Excel library.
2. For each participant file, any gaps in the data are filled in using linear interpolation first. 
3. Next, every two records are taken to calculate the labels using I-VT and I-DT and these are stored into a new dataframe. 

# Set Up Python Notebook

## Import Python Libraries

In [15]:
from os import listdir
import pandas as pd

# import spark libraries
import findspark
findspark.init()
from pyspark.sql import SparkSession

## Define Global Variables

In [16]:
datasets = ['dataset_training','dataset_testing','dataset_validation'] # directories for training, testing and valdiation
sheet = 'Gaze Data' # name of sheet with the eye tracking data

## Create Spark Session

In [17]:
spark = SparkSession.builder.appName('CIS6180_FinalProject').config("spark.jars.packages","com.crealytics:spark-excel_2.11:0.12.2").getOrCreate()


## Import the Dataset

In [19]:
# iterate through all the files in the dataset (~6-8 minutes)
for dataset in datasets:
    data_files = listdir(dataset)
    for f in data_files:
        file_path = dataset + '/' + f # file path is the relative file path for the current excel file
        print(file_path)

        # read as pandas dataframe and then convert to pyspark
        df = spark.read.format("com.crealytics.spark.excel").option("header","true").option("sheetName",sheet).load(file_path,inferSchema=True)

dataset_training/eye-data-10327.xlsx


Py4JJavaError: An error occurred while calling o68.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: com.crealytics.spark.excel. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:208)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:186)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethod